In [7]:
# Importing modules 
import sys
sys.path.append('scrape')
from scrape_linkedIn import *

import sys
sys.path.append('/')
from functions import *
from blur_and_teeth import *

In [6]:
# Unimport a module
# sys.modules.pop('blur_and_teeth')

<module 'blur_and_teeth' from '/Users/chenlang/Desktop/AI Australia/For_Git/blur_and_teeth.py'>

In [8]:
# Given the LinedIn profile url, retrieve the img url
def save_img(img_url):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))
    img_file_path = 'Saved_images/profile_pic.jpg'
    img.save(img_file_path)

In [9]:
# Creating functions to process a profile url
def process_profileURL(sample_url):
    returned_info = dict()
    # If the profile picture is empty, then catch the Exception
    try:
        img_url = get_img_url(driver, sample_url)
    except:
        print("Oops! No profile picture...")
        return returned_info

    # Save the img for later process in blur
    save_img(img_url)

    # Return the detected face
    detected_faces = get_faceDict(img_url)

    try:
        assert(len(detected_faces)>0)
    except:
        print('Oops...No face detected from image!')
        return returned_info

    # Assume that there's only one face detected in a linkIn profile
    face_dict = detected_faces[0]

    # Load the img from local disk
    img_path = "Saved_images/profile_pic.jpg"
    img = load_img(img_path)

    # Teeth detection
    teeth = detect_teeth(face_detector, img, landmark_detector)
    returned_info['teeth'] = teeth
    print('Teeth detected: ', teeth)

    # Face big enough?
    face_big = face_big_enough(face_detector, img, landmark_detector)
    returned_info['face_big'] = face_big
    print("Face big enough? ", face_big)

    # Detect emotion
    my_emotion = process_emotions(face_dict)
    returned_info['emotion'] = my_emotion
    print("Emotion: ", my_emotion)

    # Blur the background from the face
    blur_it(img, 8, face_detector, landmark_detector)
    return returned_info


In [25]:
# Set up the driver and login 
DRIVER_PATH = '/Users/chenlang/Documents/ChromeDriver/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
sign_in(driver)

In [14]:
# Change sample_url to the profile url you want.
sample_url = 'https://www.linkedin.com/in/alexcsmith/'

In [27]:
pic_info = process_profileURL(sample_url)